In [20]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [21]:
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env file
api_key = os.getenv("OPENROUTER_API_KEY")

In [23]:
from openai import OpenAI
import os

# Securely Load OpenRouter API Key
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env file
api_key = os.getenv("OPENROUTER_API_KEY")


if not api_key:
    raise ValueError("❌ ERROR: OpenRouter API key is missing. Set it as an environment variable.")

# Initialize OpenAI client with OpenRouter configuration
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

def generate_code_review(user_code, retrieved_snippets):
    """Generates a code review using the Dolphin-3 Mistral model from OpenRouter."""
    
    # Construct the review prompt with retrieved context
    context_snippets = "\n\n".join(
        f"Snippet {i+1}:\n{snippet.get('code', 'No code available')}" 
        for i, snippet in enumerate(retrieved_snippets) if isinstance(snippet, dict)
    )
    
    prompt = f"""
    You are an AI code reviewer. Analyze the following user-submitted code snippet:
    
    >>> User's Code:
    {user_code}
    
    Compare it with similar retrieved snippets below:
    
    {context_snippets}

    Identify potential issues, suggest improvements, and highlight any best practices.
    Provide clear explanations in a structured format.
    """
    
    try:
        response = client.chat.completions.create(
            model="cognitivecomputations/dolphin3.0-r1-mistral-24b:free",
            messages=[
                {"role": "system", "content": "You are an expert code reviewer."},
                {"role": "user", "content": prompt}
            ]
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"⚠️ Error calling OpenRouter: {e}"

# Example usage (unchanged)
test_code = """def add(a, b): return a + b"""
retrieved_snippets = [
    {"code": "def sum(x, y): return x + y"},
    {"code": "def addition(a, b): return a + b"}
]

review_output = generate_code_review(test_code, retrieved_snippets)
print("\n\033[92m🔍 Code Review Output:\033[0m\n", review_output)


🔍 Code Review Output:
 ### Analysis of User-Submitted Code:
**Function:** `def add(a, b): return a + b`

---

### **Potential Issues:**
1. **Name Collision with Built-in Function**  
   - The name `add` might conflict with the built-in `sum` function in Python or other libraries (e.g., `numpy.add`). While not a direct collision, it could lead to ambiguity or unintended behavior.

2. **Lack of Type Hints**  
   - The function does not include type hints for parameters or results, making it harder to understand expected input/output types for users or static analysis tools.

3. **No Error Handling**  
   - The function assumes `a` and `b` are numerical, but non-numeric inputs (e.g., strings) may produce unintended results (e.g., concatenation instead of addition).

4. **Minimal Documentation**  
   - A docstring is missing, which limits the function's usability for other developers.

5. **Generic Parameter Names**  
   - While `a` and `b` are common in mathematical contexts, more descri

In [25]:
from openai import OpenAI
import os
from retrieval import retrieve_similar_snippets  # Import retrieval function

# Securely Load OpenRouter API Key
api_key = os.getenv("OPENROUTER_API_KEY")

if not api_key:
    raise ValueError("❌ ERROR: OpenRouter API key is missing. Set it as an environment variable.")

# Initialize OpenAI client with OpenRouter configuration
client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

def generate_code_review(user_code):
    """Runs the full pipeline: retrieve similar code → generate AI review."""
    
    print("\033[92m🔍 Retrieving Similar Code Snippets...\033[0m")  # Green color
    retrieved_snippets = retrieve_similar_snippets(user_code)

    # If no similar snippets are found
    if not retrieved_snippets or isinstance(retrieved_snippets, str):
        print("\033[91m❌ No relevant code snippets found. Proceeding with AI review anyway.\033[0m")
        retrieved_snippets = []

    # Construct AI prompt with retrieved context
    context_snippets = "\n\n".join(
        f"Snippet {i+1}:\n{snippet.get('code', 'No code available')}" 
        for i, snippet in enumerate(retrieved_snippets) if isinstance(snippet, dict)
    )

    prompt = f"""
    You are an AI code reviewer. Analyze the following user-submitted code snippet:
    
    >>> User's Code:
    {user_code}
    
    Compare it with similar retrieved snippets below:
    
    {context_snippets}

    Identify potential issues, suggest improvements, and highlight best practices.
    Provide clear explanations in a structured format.
    """

    print("\033[96m🧠 Generating AI-Powered Code Review...\033[0m")  # Cyan color
    
    try:
        response = client.chat.completions.create(
            model="cognitivecomputations/dolphin3.0-r1-mistral-24b:free",  # Free OpenRouter model
            messages=[{"role": "system", "content": "You are an expert code reviewer."},
                      {"role": "user", "content": prompt}]
        )
        
        print("\n\033[92m✅ AI Code Review Output:\033[0m\n")
        return response.choices[0].message.content
    
    except Exception as e:
        return f"⚠️ Error calling OpenRouter: {e}"

# Interactive Input
user_code = input("\n💻 Enter your code snippet for review:\n")
review_output = generate_code_review(user_code)

# Display AI Review Output
print("\n", review_output)


💻 Enter your code snippet for review:
def multiply(x, y): return x * y
🔍 Retrieving Similar Code Snippets...


Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


🧠 Generating AI-Powered Code Review...

✅ AI Code Review Output:


 In this submission, the user has provided a simple python function(definition) to multiply two numbers (x, y) and return the result. The implementation is correct, but as a code reviewer, I'd like to suggest a few improvements and best practices. 

1. Error Handling: The code does not have any error handling. It does not check the type of the variables being passed into the function. This function will work as long as 'x' and 'y' are numbers, but if non-numeric values are passed in, the function will fail with a TypeError. You can add checks at the start of the function to ensure that both x and y are either type int or type float.

2. Unnecessary Return: The return statement in the function can be made more explicit by using parentheses, i.e return(x * y).

3. Docstring: A docstring is missing. A docstring provides a description of what the function does, which can be useful for other developers who may need to unders